# Notebook for text extraction on image
Inga Ulusoy, SSC, July 2022

In [ ]:
import os
from IPython.display import Image, display

In [ ]:
def list_of_files(path):
    """Get all the png files in the directory and put them in a list."""
    file_list = [f for f in os.listdir(path) if f.endswith(".png")]
    return file_list

In [ ]:
mypath = "../data/test/"
myfiles = list_of_files(mypath)

In [ ]:
for i in myfiles:
    display(Image(filename=mypath + i))
    print(i)

In [ ]:
# start with only English
mysubfiles = [i for i in myfiles if "eng" in i]

In [ ]:
for i in mysubfiles:
    display(Image(filename=mypath + i))
    print(i)

In [ ]:
# transform the image to greyscale